In [1]:
import time
import random
import os
import pandas as pd
import csv
import numpy as np
from itertools import chain
from pdp_lib import processing_google_map as proc
from pdp_lib import util
from GA_lib import GA_multi_depot as GA
from GA_lib import operation_multi_depot as operation
from GA_lib import evaluate_multi_depot as evaluate
from statistics import mode

In [2]:
def solve_and_print(filepath, rounds = 10, population_size = 100, generations=2000, crossoverRate=1.0, mutationRate=0.5):
    process_time = time.time()
    numVehicles, LoadCapacities, speed, data = proc.load_file(filepath)
    LOCATIONS = data[0]
    DEMANDS = data[1]
    timeWindows = data[2]
    serviceTimes = data[3]
    pickupSiblings = data[4]
    deliverySiblings = data[5]
    requestType = data[6]
    REQUESTS = proc.generate_request(pickupSiblings, deliverySiblings, requestType)
    DISTANCES = proc.createDistanceTable('real_map/DISTANCES.csv')
    DURATIONS = proc.createDurationTable('real_map/DURATIONS.csv')
    DEPOTS = proc.create_depots('real_map/depots.csv')
    DISTANCES_FROM_DEPOTS = proc.distances_from_depots('real_map/DISTANCES_FROM_DEPOTS.csv')
    DISTANCES_TO_DEPOTS = proc.distances_to_depots('real_map/DISTANCES_TO_DEPOTS.csv')
    # DEPOT_NUMBERS = proc.simple_assign_depots(REQUESTS, LOCATIONS, DEPOTS, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS)
    # DEPOT_NUMBERS = proc.worse2worst_assign_depots(REQUESTS, timeWindows,DISTANCES,DURATIONS,DEPOTS,DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS)
    DEPOT_NUMBERS = proc.vote_assign_depots(REQUESTS, timeWindows, DISTANCES, DURATIONS, DEPOTS, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS,k=3)
    # DEPOT_NUMBERS = proc.final_assign_depots(REQUESTS, timeWindows, DISTANCES, DURATIONS, DEPOTS, DISTANCES_FROM_DEPOTS,DISTANCES_TO_DEPOTS,k=3)
    REQ_BY_DEPOTS = proc.requests_by_depots(DEPOTS, REQUESTS, DEPOT_NUMBERS)
    print(REQ_BY_DEPOTS)
    print('----------------Instance name: ' + str(filepath) + '-----------------------------------')
    print(" processing time --- %s seconds ---" % (time.time() - process_time))

    total_results = []
    for round in range(rounds):
        results_all_depots = []
        distance_all_depots = []
        time_all_depots = []
        for id_depot in range(len(DEPOTS)):
            THIS_DEP_REQS = REQ_BY_DEPOTS[id_depot]
            NODES_OF_THIS_DEPOT = proc.locations_of_this_depot(id_depot, REQ_BY_DEPOTS, LOCATIONS)
            if(len(NODES_OF_THIS_DEPOT)==0):
                results_all_depots.append([])
                distance_all_depots.append(0)
                time_all_depots.append(0)
                continue
            # ############### SOLVING THE PROBLEMS !!!!!!!! ######################################
            pops_create_time = time.time()
            calculation_time = time.time()
            print('#################### Round :' +str(round+1)+' ############################')
            ## Initialize the populations
            populations = []
            for _ in range(population_size):
                chromosome = GA.initialize_Feasible_chromosome(DISTANCES, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS, id_depot, DURATIONS, timeWindows, THIS_DEP_REQS, DEMANDS, LoadCapacities)
                populations.append(chromosome)
            # print("Populations creation time --- %s seconds ---" % (time.time() - pops_create_time))
            ## Crossovers and mutate ##
            GA_time = time.time()

            ## Evaluate fitnesses of all populations ##
            best_fitness_so_far = -99999999
            for gen in range(generations):
                fitness_table = []
                for chromosome in populations:
                    fitness_table.append(
                        evaluate.chromosomeFitness(chromosome, DISTANCES, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS,
                                                   id_depot))
                populations = [x for _, x in sorted(zip(fitness_table, populations), reverse=True)]
                populations.pop()
                populations.pop()
                elite1 = populations.pop(0)
                elite2 = populations.pop(0)


                ## Tournament Selection ##
                # id1, id2 = random.randrange(0, len(populations)), random.randrange(0, len(populations))
                id1,id2,id3,id4 = random.sample(range(0, len(populations)), 4)
                fitness1,fitness2,fitness3,fitness4 = fitness_table[id1],fitness_table[id2],fitness_table[id3],fitness_table[id4]
                candidate1,candidate2,candidate3,candidate4 = populations[id1], populations[id2],populations[id3],populations[id4]
                parent1 = operation.tournament_selection(candidate1,candidate2,fitness1,fitness2)
                parent2 = operation.tournament_selection(candidate3,candidate4,fitness3,fitness4)

                ## Crossing over !! ####
                child1, child2 = operation.crossover(DISTANCES, DURATIONS, timeWindows, THIS_DEP_REQS, parent1, parent2,
                                                     DEMANDS, LoadCapacities, DISTANCES_FROM_DEPOTS,
                                                     DISTANCES_TO_DEPOTS, id_depot, crossoverRate)
                # if(not evaluate.haveEqualNodes(child1,child2,LOCATIONS)):
                #         print('note have Equal nodes, Crossover Bug!!!!!'+ str(gen))
                #         break
                #
                ## Mutation ##
                child1 = operation.mutate(child1, DISTANCES, DURATIONS, timeWindows, THIS_DEP_REQS, DEMANDS,
                                          LoadCapacities, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS, id_depot, mutationRate)
                child2 = operation.mutate(child2, DISTANCES, DURATIONS, timeWindows, THIS_DEP_REQS, DEMANDS,
                                          LoadCapacities, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS, id_depot, mutationRate)
                if (not evaluate.haveEqualNodes(child1, child2, NODES_OF_THIS_DEPOT)):
                    print('note have Equal nodes, Mutation Bug!!!!!' + str(gen))
                    break


                ## Evaluate Fitness of the best ones ###
                current_fitness = evaluate.chromosomeFitness(child1, DISTANCES, DISTANCES_FROM_DEPOTS,
                                                             DISTANCES_TO_DEPOTS, id_depot)
                current_fitness = max(current_fitness,
                                      evaluate.chromosomeFitness(child2, DISTANCES, DISTANCES_FROM_DEPOTS,
                                                                 DISTANCES_TO_DEPOTS, id_depot))
                current_fitness = max(current_fitness,
                                      evaluate.chromosomeFitness(elite1, DISTANCES, DISTANCES_FROM_DEPOTS,
                                                                 DISTANCES_TO_DEPOTS, id_depot))
                current_fitness = max(current_fitness,
                                      evaluate.chromosomeFitness(elite2, DISTANCES, DISTANCES_FROM_DEPOTS,
                                                                 DISTANCES_TO_DEPOTS, id_depot))

                ## Add them back into populations ###
                populations.append(child1)
                populations.append(child2)
                populations.append(elite1)
                populations.append(elite2)

                ### Check break conditions ####
                if (current_fitness > best_fitness_so_far):
                    best_fitness_so_far = current_fitness
                    bestFitGen = gen
                    # print('#### New Best Fitness !! , Best so far is :' + str(10000.0 / best_fitness_so_far) + '#####')
                    # print('#### This Generation: ' + str(gen) + '#######')
                if (gen - bestFitGen >= 1000):
                    # print('#### Break Generation: ' + str(gen) + '#######')
                    break
                # print('############# Round-'+str(round+1) +',DEPOT-'+str(id_depot)+','+ 'Generation:' + str(gen + 1) + ' #########################')
                # print('This Gen Distance :' + str(10000.0 / current_fitness))

            ### Evaluate results of current DEPOT ####
            fitness_table = []
            for chromosome in populations:
                fitness_table.append(
                    evaluate.chromosomeFitness(chromosome, DISTANCES, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS, id_depot))
            populations = [x for _, x in sorted(zip(fitness_table, populations), reverse=True)]
            # print("GA time --- %s seconds ---" % (time.time() - GA_time))
            best_chromosome = populations[0]
            computational_time = time.time() - calculation_time
            # print("Total Calculation time --- %s seconds ---" % (computational_time))
            distance = evaluate.chromosomeRoutesDistance(best_chromosome,DISTANCES,DISTANCES_FROM_DEPOTS,DISTANCES_TO_DEPOTS,id_depot)
            # print('Distances of the best chromosome: ' + str(distance))
            # print(best_chromosome)

            ## Memo the results ##
            results_all_depots.append(best_chromosome)
            distance_all_depots.append(distance)
            time_all_depots.append(computational_time)
        total_results.append((results_all_depots,distance_all_depots,time_all_depots))
        print(results_all_depots)
    return total_results

In [3]:
def mean_results(results):
    solutions = []
    cal_times = []
    distances = []
    for round, round_result in enumerate(results):
        sol = results[round][0]
        dist = results[round][1]
        ct = results[round][2]

        solutions.append(sol)
        distances.append(dist)
        cal_times.append(ct)
    distances = np.array(distances)
    distances = pd.DataFrame(distances)
    distances = np.mean(distances, axis=0)
    cal_times = np.array(cal_times)
    cal_times = pd.DataFrame(cal_times)
    cal_times = np.mean(cal_times, axis=0)
    return solutions,distances, cal_times


In [4]:
def create_worst_case(filepath, rounds = 1, population_size = 100, generations=3000, crossoverRate=1.0, mutationRate=0.5):
    process_time = time.time()
    numVehicles, LoadCapacities, speed, data = proc.load_file(filepath)
    LOCATIONS = data[0]
    DEMANDS = data[1]
    timeWindows = data[2]
    serviceTimes = data[3]
    pickupSiblings = data[4]
    deliverySiblings = data[5]
    requestType = data[6]
    REQUESTS = proc.generate_request(pickupSiblings, deliverySiblings, requestType)
    DISTANCES = proc.createDistanceTable('real_map/DISTANCES.csv')
    DURATIONS = proc.createDurationTable('real_map/DURATIONS.csv')
    DEPOTS = proc.create_depots('real_map/depots.csv')
    DISTANCES_FROM_DEPOTS = proc.distances_from_depots('real_map/DISTANCES_FROM_DEPOTS.csv')
    DISTANCES_TO_DEPOTS = proc.distances_to_depots('real_map/DISTANCES_TO_DEPOTS.csv')
    DEPOT_NUMBERS = proc.simple_assign_depots(REQUESTS, LOCATIONS, DEPOTS, DISTANCES_FROM_DEPOTS, DISTANCES_TO_DEPOTS)
    REQ_BY_DEPOTS = proc.requests_by_depots(DEPOTS, REQUESTS, DEPOT_NUMBERS)
    
    res = GA.initialize_WorstCase_Chromosome(REQUESTS)
    
    
    dist_worst = 0
    for [idx,_,[p,d]] in res:
        depot = int(DEPOT_NUMBERS[idx]) 
        dist_worst += (DISTANCES_FROM_DEPOTS[depot][p]+DISTANCES[p][d]+DISTANCES_TO_DEPOTS[d][depot])

    return res,dist_worst

# Automated run

In [25]:
path = 'pdp_instances/GOO/doing/'
FILENAMES = next(os.walk(path))[2]

all_results = []

for name in FILENAMES:
    filepath = path + name
    results = solve_and_print(filepath)
    solutions,distances,cal_times = mean_results(results)
    all_results.append((solutions,distances,cal_times))

[{10: (28, 5), 12: (33, 15), 15: (39, 11), 17: (47, 19), 18: (48, 6), 22: (53, 45), 28: (61, 44), 29: (62, 38), 32: (65, 46), 33: (66, 79), 35: (73, 42), 36: (74, 43), 37: (75, 34), 38: (76, 30), 39: (80, 26)}, {21: (52, 27)}, {0: (3, 13), 1: (4, 7), 2: (8, 32), 3: (9, 16), 4: (17, 50), 6: (20, 1), 8: (24, 22), 9: (25, 23), 11: (29, 35), 16: (40, 31), 31: (64, 12)}, {}, {5: (18, 78), 7: (21, 57), 13: (36, 69), 14: (37, 77), 19: (49, 72), 20: (51, 68), 23: (54, 60), 24: (55, 67), 25: (56, 2), 26: (58, 10), 27: (59, 14), 30: (63, 70), 34: (71, 41)}]
----------------Instance name: pdp_instances/GOO/doing/goo10.csv-----------------------------------
 processing time --- 0.37502121925354004 seconds ---
#################### Round :1 ############################


#################### Round :1 ############################


#################### Round :1 ############################


#################### Round :1 ############################


[[[3, [39, 32, 17, 15, 38, 10], [80, 47, 39, 28, 26, 19, 11, 5, 65, 76, 46, 30]], [2, [37, 29, 12, 36], [62, 75, 74, 38, 34, 33, 15, 43]], [0, [18, 33, 35, 22, 28], [48, 66, 6, 79, 53, 73, 61, 45, 44, 42]]], [[0, [21], [52, 27]]], [[2, [9, 6, 4, 3], [20, 25, 23, 17, 1, 50, 9, 16]], [1, [8, 11, 16, 2], [8, 29, 35, 40, 32, 31, 24, 22]], [0, [31, 1, 0], [4, 3, 7, 64, 13, 12]]], [], [[2, [34, 20, 13, 24, 23], [71, 41, 36, 51, 54, 55, 60, 69, 67, 68]], [0, [30, 19, 25, 26, 27, 5, 7, 14], [58, 59, 37, 10, 14, 63, 77, 56, 70, 2, 18, 21, 49, 78, 57, 72]], [1, [], []]]]
#################### Round :2 ############################


...

#################### Round :10 ############################


#################### Round :10 ############################


#################### Round :10 ############################


#################### Round :10 ############################


[[[3, [29], [62, 80]], [2, [19, 20, 21, 14, 16, 13, 12], [33, 44, 43, 45, 32, 30, 36, 37, 34, 39, 31, 18, 14, 9]], [1, [28, 15, 18, 22], [38, 48, 42, 61, 5, 11, 67, 7]]], [[0, [36, 26, 25], [57, 56, 72, 47, 27, 28]]], [[3, [3, 8, 2, 7, 31, 1, 5, 9, 35, 0], [8, 4, 64, 70, 65, 13, 12, 6, 1, 3, 21, 23, 22, 49, 24, 20, 41, 35, 17, 2]], [2, [24, 4, 38, 33, 6, 32, 30, 34, 10], [10, 25, 63, 78, 54, 55, 68, 26, 19, 16, 15, 50, 69, 66, 71, 73, 58, 46]]], [[0, [37], [76, 52]]], [[1, [39, 17], [79, 40, 74, 53]], [0, [27, 23, 11], [29, 51, 59, 60, 75, 77]]]]


# The GOO Instances

# Average list by columns

In [26]:
def count_num_vehicle(solutions):
    nums = []
    for sol in solutions:
        sol = [x for x in sol if len(x[1])>0]
        nums.append(len(sol))
    return nums

In [27]:
# res =results[0]
# SOL,DIST,CT = res[0],res[1],res[2]


In [28]:
# worst_sol,distance_worst = create_worst_case('pdp_instances/GOO/goo12.csv')
# distance_worst

In [29]:
# SOL[2]
# count_num_vehicle([worst_sol])

# Writing to CSV

In [30]:
solutions,distances,cal_times = mean_results(all_results)
dist = [dist for _,dist,_ in all_results]
dist = np.array(dist)
df = pd.DataFrame(dist)
df.to_csv('distances_goo2-5.csv')
CT = [ct for _,_,ct in all_results]
CT = np.array(CT)
df_CT = pd.DataFrame(CT)
df_CT.to_csv('CT_goo2-5.csv')

# Average list by columns

In [31]:
def count_num_vehicle(solutions):
    nums = []
    for sol in solutions:
        sol = [x for x in sol if len(x[1])>0]
        nums.append(len(sol))
    return nums

In [32]:

# worst_sol

In [33]:
def avg_table(table):
    transposed = zip(*table)
    avg = lambda items: float(sum(items)) / len(items)
    averages = map(avg, transposed)
    averages = list(averages)
    return  averages

In [34]:
# for sol in solutions:
num_vehicles  = []
i = 0
for res in all_results:
    solutions = res[0]
    arr_temp = []
    for sol in solutions:
        num_temp = count_num_vehicle(sol)
        arr_temp.append(num_temp)
    avg_temp = avg_table(arr_temp)
    print(avg_temp)
    num_vehicles.append(avg_temp)

num_vehicles = np.array(num_vehicles)
df_num_vehicles = pd.DataFrame(num_vehicles)
df_num_vehicles.to_csv('num_veh_goo.csv')

[3.1, 1.0, 2.9, 0.0, 2.0]
[3.0, 0.0, 2.9, 0.0, 2.0]
[4.0, 0.0, 4.3, 0.0, 3.0]
[2.0, 0.0, 4.9, 0.0, 4.2]
[2.0, 1.0, 3.5, 0.0, 2.0]
[3.0, 0.0, 5.0, 0.0, 3.0]
[2.9, 1.0, 2.8, 1.0, 2.0]


In [35]:
# results = [([[[2,[16,19],[41,37,34,22]],[1,[13,4,6],[15,31,43,18,48,42]],[0,[15],[35,49]]],[[1,[39,25,24],[52,51,80,21,5,10]],[0,[38],[79,61]]],[[2,[23,7,21],[19,47,45,50,53,25]],[5,[12,31,18],[62,40,30,23,20,13]],[3,[2,32,14,11,27,10,33],[32,7,4,29,33,63,55,77,69,28,27,64,71,65]],[1,[],[]],[0,[34,9,8,0],[3,26,24,68,67,66,2,14]],[4,[3,5],[9,16,12,11]]],[[0,[30],[60,58]]],[[3,[37,35,36],[74,75,73,76,17,8]],[2,[],[]],[1,[29,28,1],[56,57,38,1,6,59]],
                                                                                                                                                                                                                                                                                                                                                                                       [0,[26,22,20,17],[54,46,44,36,39,70,78,72]]]])]

IndentationError: unexpected indent (<ipython-input-35-06f82f746798>, line 2)

In [23]:
# res = results[0]

In [24]:
# all_results

In [24]:
# 
# num_vehicles = np.array(num_vehicles)
# df_num_vehicles = pd.DataFrame(num_vehicles)
# df_num_vehicles.to_csv('num_veh_goo.csv')

In [ ]:
## Run
path = 'pdp_instances/GOO/doing/'
FILENAMES = next(os.walk(path))[2]

all_results = []

for name in FILENAMES:
    filepath = path + name
    results = solve_and_print(filepath)
    solutions,distances,cal_times = mean_results(results)
    all_results.append((solutions,distances,cal_times))
    
# count veh
def count_num_vehicle(solutions):
    nums = []
    for sol in solutions:
        sol = [x for x in sol if len(x[1])>0]
        nums.append(len(sol))
    return nums

solutions,distances,cal_times = mean_results(all_results)
dist = [dist for _,dist,_ in all_results]
dist = np.array(dist)
df = pd.DataFrame(dist)
df.to_csv('distances_goo-3-vote.csv')
CT = [ct for _,_,ct in all_results]
CT = np.array(CT)
df_CT = pd.DataFrame(CT)
df_CT.to_csv('CT_goo-2-vote.csv')

def avg_table(table):
    transposed = zip(*table)
    avg = lambda items: float(sum(items)) / len(items)
    averages = map(avg, transposed)
    averages = list(averages)
    return  averages

# for sol in solutions:
num_vehicles  = []
i = 0
for res in all_results:
    solutions = res[0]
    arr_temp = []
    for sol in solutions:
        num_temp = count_num_vehicle(sol)
        arr_temp.append(num_temp)
    avg_temp = avg_table(arr_temp)
    print(avg_temp)
    num_vehicles.append(avg_temp)

num_vehicles = np.array(num_vehicles)
df_num_vehicles = pd.DataFrame(num_vehicles)
df_num_vehicles.to_csv('num_veh_3-vote.csv')

[{4: (15, 43), 6: (18, 42), 12: (30, 23), 13: (31, 48), 14: (32, 4), 15: (35, 49), 16: (37, 22), 18: (40, 13), 19: (41, 34), 21: (45, 25), 23: (47, 50), 31: (62, 20)}, {24: (51, 5), 25: (52, 21), 38: (79, 61), 39: (80, 10)}, {0: (3, 66), 1: (6, 59), 2: (7, 33), 3: (9, 12), 5: (16, 11), 7: (19, 53), 8: (24, 2), 9: (26, 68), 10: (28, 71), 11: (29, 69), 27: (55, 27), 32: (63, 77), 33: (64, 65), 34: (67, 14)}, {}, {17: (39, 72), 20: (44, 78), 22: (46, 70), 26: (54, 36), 28: (56, 38), 29: (57, 1), 30: (60, 58), 35: (74, 73), 36: (75, 17), 37: (76, 8)}]
----------------Instance name: pdp_instances/GOO/doing/goo1.csv-----------------------------------
 processing time --- 0.3400437831878662 seconds ---
#################### Round :1 ############################


#################### Round :1 ############################


#################### Round :1 ############################


#################### Round :1 ############################


[[[3, [31, 6, 12, 23, 18], [62, 47, 40, 30, 23, 20, 18, 13, 50, 42]], [1, [15], [35, 49]], [0, [], []], [2, [16, 19, 14, 4, 13, 21], [32, 15, 4, 31, 41, 43, 48, 45, 37, 34, 22, 25]]], [[1, [38], [79, 61]], [0, [39, 25, 24], [52, 51, 80, 21, 5, 10]]], [[4, [34, 9, 8, 0], [3, 26, 24, 68, 67, 66, 2, 14]], [3, [1, 7], [6, 19, 59, 53]], [0, [32, 11, 10, 2, 27, 33], [7, 29, 33, 63, 55, 77, 69, 28, 27, 64, 71, 65]], [1, [5, 3], [9, 16, 12, 11]]], [], [[3, [37, 35, 36], [74, 75, 73, 76, 17, 8]], [2, [29, 28], [56, 57, 38, 1]], [1, [30], [60, 58]], [0, [20, 22, 17, 26], [54, 46, 44, 36, 39, 70, 78, 72]]]]
#################### Round :2 ############################


...

#################### Round :4 ############################


[[[2, [31, 23, 18, 12, 6], [62, 47, 40, 30, 23, 20, 18, 13, 50, 42]], [1, [15], [35, 49]], [0, [14, 4, 13, 19, 16, 21], [32, 15, 4, 31, 41, 43, 48, 45, 37, 34, 22, 25]]], [[1, [39, 24, 25], [52, 51, 80, 21, 5, 10]], [0, [38], [79, 61]]], [[2, [5, 3], [9, 16, 12, 11]], [1, [11, 32, 2, 10, 27, 33], [7, 29, 33, 63, 55, 77, 69, 28, 27, 64, 71, 65]], [3, [1, 7], [6, 19, 59, 53]], [0, [34, 9, 8, 0], [3, 26, 24, 68, 67, 66, 2, 14]]], [], [[3, [37, 35, 36], [74, 75, 73, 76, 17, 8]], [2, [30], [60, 58]], [1, [29, 28], [56, 57, 38, 1]], [0, [26, 22, 20, 17], [54, 46, 44, 36, 39, 70, 78, 72]]]]
#################### Round :5 ############################


#################### Round :5 ############################


#################### Round :5 ############################


#################### Round :5 ############################


{(1, 2),
 (3, 35),
 (4, 64),
 (8, 13),
 (12, 24),
 (20, 17),
 (23, 41),
 (25, 16),
 (63, 50),
 (65, 21),
 (66, 46),
 (70, 6),
 (72, 28)}

In [82]:
worst_sol2 = [ x for x in worst_sol if tuple(x[2]) in s]

In [83]:
worst_sol2

[]